In [25]:
import os
import rasterio
import numpy as np

def getFileList(dir,Filelist, ext=None):
    """
    获取文件夹及其子文件夹中文件列表
    输入 dir：文件夹根目录
    输入 ext: 扩展名
    返回： 文件路径列表
    """
    newDir = dir
    if os.path.isfile(dir):
        if ext is None:
            Filelist.append(dir)
        else:
            if ext in dir[-3:]:
                Filelist.append(dir)
    
    elif os.path.isdir(dir):
        for s in os.listdir(dir):
            newDir=os.path.join(dir,s)
            getFileList(newDir, Filelist, ext)
 
    return Filelist

    
filelist=getFileList(r'D:\饱和土壤水\SSP585_C',[],'tif')
# for i in filelist:
#     print(i)
#     print(i.split('\\')[-1],i.split('\\')[-1][:4])
#     print('\n')

In [26]:
for j in range(2015,2101):
    years = []
    for i in filelist:
        if str(j) in i.split('\\')[-1][:4]:
            years.append(i)
#     print(years)
    dataset_0 = rasterio.open(years[0])
    width=dataset_0.width
    height=dataset_0.height
    array=np.zeros((width*height,len(years)))
    array.shape
    for i in range(len(years)):

        dataset = rasterio.open(years[i])
        band1 = dataset.read(1)                                                                                                                                        
        band1[np.abs(band1)>100]=np.nan
        array[:,i]=band1.reshape(width*height)
    arraymean=np.mean(array,axis=1)
    with rasterio.open(
        r'D:\饱和土壤水\SSP585\RSM000050_SSP585_{}.tif'.format(j),
        'w',
         driver='GTiff',
         height=height,
         width=width,
         count=1,
         dtype=dataset.dtypes[0],
         crs=dataset.crs,
         transform=dataset.transform,
    ) as dst:
        dst.write(arraymean.reshape((height,width)), 1)